In [ ]:
import requests
import psycopg2
from time import sleep
from datetime import datetime

In [10]:
# Jikan API base URL and delay
JIKAN_BASE_URL = "https://api.jikan.moe/v4"
RATE_LIMIT_DELAY = 1.5  # seconds

In [26]:
# PostgreSQL connection
conn = psycopg2.connect(
    dbname="anime_db",
    user="root",
    password="root",
    host="localhost"
)
cursor = conn.cursor()

In [27]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS anime (
    anime_id INT PRIMARY KEY,
    title TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS staff (
    staff_id INT PRIMARY KEY,
    name TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS staff_roles (
    anime_id INT,
    staff_id INT,
    role TEXT,
    PRIMARY KEY (anime_id, staff_id, role),
    FOREIGN KEY (anime_id) REFERENCES anime(anime_id),
    FOREIGN KEY (staff_id) REFERENCES staff(staff_id),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS anime_ratings (
    anime_id INT PRIMARY KEY,
    rating FLOAT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (anime_id) REFERENCES anime(anime_id)
);
""")

conn.commit()


In [28]:
def get_anime_details(anime_id):
    """Fetch anime title and rating using Jikan"""
    try:
        response = requests.get(f"{JIKAN_BASE_URL}/anime/{anime_id}")
        response.raise_for_status()
        data = response.json().get("data", {})
        title = data.get("title")
        rating = data.get("score")  # Might be None
        return title, rating
    except Exception as e:
        print(f"[!] Failed to get anime details {anime_id}: {e}")
        return f"Anime {anime_id}", None


In [29]:
def get_anime_staff(anime_id):
    """Fetch staff data for a specific anime from Jikan API"""
    try:
        response = requests.get(f"{JIKAN_BASE_URL}/anime/{anime_id}/staff")
        response.raise_for_status()
        return response.json().get("data", [])
    except Exception as e:
        print(f"[!] Failed to get staff for anime {anime_id}: {e}")
        return []

In [30]:
def insert_data(conn, anime_id, title, staff_data, rating):
    with conn.cursor() as cursor:
        try:
            # Insert anime
            cursor.execute("""
                INSERT INTO anime (anime_id, title)
                VALUES (%s, %s)
                ON CONFLICT (anime_id) DO UPDATE SET
                    title = EXCLUDED.title,
                    updated_at = CURRENT_TIMESTAMP;
            """, (anime_id, title))

            # Insert anime rating
            cursor.execute("""
                INSERT INTO anime_ratings (anime_id, rating)
                VALUES (%s, %s)
                ON CONFLICT (anime_id) DO UPDATE SET
                    rating = EXCLUDED.rating,
                    updated_at = CURRENT_TIMESTAMP;
            """, (anime_id, rating))

            # Insert staff and roles
            for staff in staff_data:
                staff_id = staff["person"]["mal_id"]
                name = staff["person"]["name"]
                roles = staff.get("positions", [])

                cursor.execute("""
                    INSERT INTO staff (staff_id, name)
                    VALUES (%s, %s)
                    ON CONFLICT (staff_id) DO UPDATE SET
                        name = EXCLUDED.name,
                        updated_at = CURRENT_TIMESTAMP;
                """, (staff_id, name))

                for role in roles:
                    cursor.execute("""
                        INSERT INTO staff_roles (anime_id, staff_id, role)
                        VALUES (%s, %s, %s)
                        ON CONFLICT (anime_id, staff_id, role) DO NOTHING;
                    """, (anime_id, staff_id, role))

            conn.commit()
            print(f"[✓] Inserted anime {anime_id}: {title} with rating {rating}")

        except Exception as e:
            conn.rollback()
            print(f"[✗] Error inserting anime {anime_id}: {e}")


In [31]:
def main(conn):
    START_ID = 1
    END_ID = 100

    try:
        for anime_id in range(START_ID, END_ID + 1):
            staff_data = get_anime_staff(anime_id)
            if not staff_data:
                continue

            title, rating = get_anime_details(anime_id)
            insert_data(conn, anime_id, title, staff_data, rating)
            sleep(RATE_LIMIT_DELAY)

    except Exception as e:
        print(f"[!] Fatal error: {e}")


In [32]:
main(conn)


[✓] Inserted anime 1: Cowboy Bebop with rating 8.75
[!] Failed to get staff for anime 2: 404 Client Error: Not Found for url: https://api.jikan.moe/v4/anime/2/staff
[!] Failed to get staff for anime 3: 404 Client Error: Not Found for url: https://api.jikan.moe/v4/anime/3/staff
[!] Failed to get staff for anime 4: 404 Client Error: Not Found for url: https://api.jikan.moe/v4/anime/4/staff
[✓] Inserted anime 5: Cowboy Bebop: Tengoku no Tobira with rating 8.38
[✓] Inserted anime 6: Trigun with rating 8.22
[✓] Inserted anime 7: Witch Hunter Robin with rating 7.24
[✓] Inserted anime 8: Bouken Ou Beet with rating 6.93
[!] Failed to get staff for anime 9: 404 Client Error: Not Found for url: https://api.jikan.moe/v4/anime/9/staff
[!] Failed to get staff for anime 10: 404 Client Error: Not Found for url: https://api.jikan.moe/v4/anime/10/staff
[!] Failed to get staff for anime 11: 404 Client Error: Not Found for url: https://api.jikan.moe/v4/anime/11/staff
[!] Failed to get staff for anime 12:

In [33]:
conn.close()
